In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical


train_df = pd.read_csv('UNSW_NB15_training-set.csv')
test_df = pd.read_csv('UNSW_NB15_testing-set.csv')


target = 'attack_cat'


drop_cols = ['id', 'label']
X_train = train_df.drop(columns=[target] + drop_cols, errors='ignore')
X_test = test_df.drop(columns=[target] + drop_cols, errors='ignore')


X_train.fillna('missing', inplace=True)
X_test.fillna('missing', inplace=True)


X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)


X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)


le_target = LabelEncoder()
y_train = le_target.fit_transform(train_df[target].fillna('None'))
y_test = le_target.transform(test_df[target].fillna('None'))


y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


X_train_cnn = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_cnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)


model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train_cat.shape[1], activation='softmax'))  # multi-class classification


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit(X_train_cnn, y_train_cat, epochs=20, batch_size=128, validation_split=0.2)


test_loss, test_acc = model.evaluate(X_test_cnn, y_test_cat)
print(f"\nTest Accuracy (Keras evaluation): {test_acc*100:.2f}%")


y_pred_probs = model.predict(X_test_cnn)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test_cat, axis=1)


accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f"\nAccuracy (sklearn): {accuracy*100:.2f}%")


f1_macro = f1_score(y_true_classes, y_pred_classes, average='macro')
f1_weighted = f1_score(y_true_classes, y_pred_classes, average='weighted')
print(f"F1 Score (Macro): {f1_macro:.4f}")
print(f"F1 Score (Weighted): {f1_weighted:.4f}")


roc_auc = roc_auc_score(y_test_cat, y_pred_probs, multi_class='ovr')
print(f"ROC AUC Score (OVR): {roc_auc:.4f}")


print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=le_target.classes_))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.6954 - loss: 0.8495 - val_accuracy: 0.8981 - val_loss: 0.3172
Epoch 2/20
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7602 - loss: 0.6267 - val_accuracy: 0.9000 - val_loss: 0.2963
Epoch 3/20
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7665 - loss: 0.5975 - val_accuracy: 0.8965 - val_loss: 0.2960
Epoch 4/20
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.7700 - loss: 0.5776 - val_accuracy: 0.8947 - val_loss: 0.2906
Epoch 5/20
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7740 - loss: 0.5668 - val_accuracy: 0.9015 - val_loss: 0.2851
Epoch 6/20
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7760 - loss: 0.5626 - val_accuracy: 0.8980 - val_loss: 0.2851
Epoch 7/20
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7798 - loss: 0.5513 - val_accuracy: 0.8972 - val_loss: 0.2796
Epoch 8/20
1096/1096 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7782 - loss: 0.5512 -